In [12]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL without the page number
base_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711368896&srchtxt=%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E6%A1%88"

# Define the CSV file to write data to
csv_file = "scraped_data.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL'])  # Writing the headers

    # Modify this range to scrape more pages
    for page_number in range(1, 26):
        # Modify the URL to include the current page number
        url = f"{base_url}&&page={page_number}"
        
        # Send a GET request to the page
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'  # Ensure correct encoding

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the elements containing the data you're interested in.
        posts = soup.find_all('li', class_='pbw')

        # Loop through each post and extract data
        for post in posts:
            # Find the title and URL
            title_link = post.find('a')
            title = title_link.get_text(strip=True) if title_link else "N/A"
            url = title_link['href'] if title_link else "N/A"
            
            # The replies and views data
            reply_view_element = post.find('p', class_='xg1')
            replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
            
            # Attempt to capture the date from the correct span element
            # We'll look for the span element that contains the date based on the format we expect
            date_spans = reply_view_element.find_all('span') if reply_view_element else []
            time = "N/A"
            for span in date_spans:
                if '-' in span.get_text():  # A simple check to see if the span might contain a date
                    time = span.get_text(strip=True)
                    break

            # Write the extracted data to the CSV
            writer.writerow([title, replies_views, time, url])

print("Data scraping and CSV writing completed.")

Data scraping and CSV writing completed.


In [4]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape individual post details
def scrape_post_details(post_url):
    response = requests.get(post_url, headers=headers)
    post_soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting the username
    username_tag = post_soup.find('a', class_='xw1')
    username = username_tag.get_text(strip=True) if username_tag else "N/A"
    
    # Extracting the main content
    main_content_tag = post_soup.find('div', id=lambda x: x and x.startswith('postmessage_'))
    main_content = main_content_tag.get_text(strip=True) if main_content_tag else "N/A"
    
    # Extracting all replies
    replies_tags = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))
    all_replies = [reply.get_text(strip=True) for reply in replies_tags[1:]] if replies_tags else []  # Skip the first one
    
    return username, main_content, all_replies


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

base_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711368896&srchtxt=%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E6%A1%88"
csv_file = "scraped_data2.csv"

# CSV file setup
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Update headers

    # Loop to go through all search result pages
    page_number = 1
    while True:
        url = f"{base_url}&page={page_number}"
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.content, 'html.parser')
        
        posts = soup.find_all('li', class_='pbw')
        if not posts:
            break  # No more posts found
        
        for post in posts:
            title_link = post.find('a')
            title = title_link.get_text(strip=True) if title_link else "N/A"
            post_url = title_link['href'] if title_link else "N/A"
            
            reply_view_element = post.find('p', class_='xg1')
            replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
            time = reply_view_element.find('span').get_text(strip=True) if reply_view_element and reply_view_element.find('span') else "N/A"
            
            # Scrape individual post details
            if post_url.startswith('/'):
                post_url = f"https://www.baby-kingdom.com{post_url}"
            username, main_content, all_replies = scrape_post_details(post_url)

            writer.writerow([title, replies_views, time, post_url, username, main_content, ' | '.join(all_replies)])

        print(f"Finished page {page_number}")
        page_number += 1

print("Scraping completed and data written to CSV.")


Finished page 1
Finished page 2
Finished page 3


KeyboardInterrupt: 

In [11]:
import requests
from bs4 import BeautifulSoup
import csv

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to scrape individual post details
def scrape_post_details(post_url):
    response = requests.get(post_url, headers=headers)
    post_soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting the username
    username_tag = post_soup.find('div', class_='authi').find('a')
    username = username_tag.get_text(strip=True) if username_tag else "N/A"
    
    # Extracting the time
    time_tag = post_soup.find('em', id=lambda x: x and x.startswith('authorposton')).find('span')
    time = time_tag['title'] if time_tag and 'title' in time_tag.attrs else "N/A"
    
    # Extracting the main content
    main_content_tag = post_soup.find('div', class_='pcb').find('div', class_='t_fsz')
    main_content = main_content_tag.get_text(strip=True) if main_content_tag else "N/A"
    
    # Extracting replies
    replies_tags = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))
    replies_text = [reply.get_text(strip=True) for reply in replies_tags[1:]] if replies_tags else []  # Skip the first one which is main content

    return username, time, main_content, replies_text

# Base URL pattern
base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711368896&srchtxt=%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E6%A1%88"

# CSV file setup
with open('scraped_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Headers

    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        
        if response.status_code != 200:
            print(f"Failed to retrieve data for page {page_number}, status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        posts = soup.find_all('li', class_='pbw')

        if not posts:
            print(f"No more topics found on page {page_number}. Ending scrape.")
            break

        for post in posts:
            title_link = post.find('a')
            title = title_link.get_text(strip=True) if title_link else "N/A"
            post_url = title_link['href'] if title_link else "N/A"

            reply_view_element = post.find('p', class_='xg1')
            replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
            
            # Scrape details from the post's individual page
            if post_url.startswith('/'):
                post_url = f"https://www.baby-kingdom.com{post_url}"
            username, time, main_content, all_replies = scrape_post_details(post_url)

            writer.writerow([title, replies_views, time, post_url, username, main_content, ' | '.join(all_replies)])

        print(f"Finished scraping page {page_number}.")
        page_number += 1

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.


AttributeError: 'NoneType' object has no attribute 'find'

In [18]:
# below is the successful script for 騙案 in BK

In [17]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies
        replies_tags = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))
        replies_text = [reply.get_text(strip=True) for reply in replies_tags[1:]] if replies_tags else []

        return username, time, main_content, replies_text
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "scraped_data4.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711368896&srchtxt=%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E6%A1%88"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, ' | '.join(all_replies)])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
Finished scraping page 11.
Finished scraping page 12.
Finished scraping page 13.
Finished scraping page 14.
Finished scraping page 15.
Finished scraping page 16.
Finished scraping page 17.
Finished scraping page 18.
Finished scraping page 19.
Finished scraping page 20.
Finished scraping page 21.
Finished scraping page 22.
Finished scraping page 23.
Finished scraping page 24.
Finished scraping page 25.
No more topics found on page 26. Ending scrape.
Data scraping and CSV writing completed.


In [21]:
#above ok, but only can scrape first page comments

In [30]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "scraped_data_with_replies.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711368896&srchtxt=%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E6%A1%88"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
Finished scraping page 11.
Finished scraping page 12.
Finished scraping page 13.
Finished scraping page 14.
Finished scraping page 15.
Finished scraping page 16.
Finished scraping page 17.
Finished scraping page 18.
Finished scraping page 19.
Finished scraping page 20.
Finished scraping page 21.
Finished scraping page 22.
Finished scraping page 23.
Finished scraping page 24.
Finished scraping page 25.
No more topics found on page 26. Ending scrape.
Data scraping and CSV writing completed.


In [31]:
#above successful

In [32]:
#now next keyword:facebook騙案

In [33]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "facebook騙案.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711979377&srchtxt=facebook+%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=facebook+%E9%A8%99%E6%A1%88&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [34]:
# scam

In [35]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "scam.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711979650&srchtxt=scam&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=scam&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [36]:
#行騙

In [37]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_行騙.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711979851&srchtxt=%E8%A1%8C%E9%A8%99&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E8%A1%8C%E9%A8%99&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
No more topics found on page 4. Ending scrape.
Data scraping and CSV writing completed.


In [38]:
#呃人

In [39]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_呃人.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711980233&srchtxt=%E5%91%83%E4%BA%BA&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E5%91%83%E4%BA%BA&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
Finished scraping page 11.
Request failed with error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Retrying...
Request failed with error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Retrying...
Request failed with error Response ended prematurely. Retrying...
Finished scraping page 12.
Request failed with error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Retrying...
Finished scraping page 13.
Finished scraping page 14.
Finished scraping page 15.
Request failed with error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Retrying...
Request fa

KeyboardInterrupt: 

In [40]:
# above stopped at p16, restart at p17

In [42]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_呃人Part2.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711980233&srchtxt=%E5%91%83%E4%BA%BA&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E5%91%83%E4%BA%BA&&page=17"

    # Start from page 17
    page_number = 17

    # Scrape process
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 17.
Finished scraping page 18.
Finished scraping page 19.
Finished scraping page 20.
Finished scraping page 21.
Finished scraping page 22.
Finished scraping page 23.
Finished scraping page 24.
Finished scraping page 25.
Finished scraping page 26.
No more topics found on page 27. Ending scrape.
Data scraping and CSV writing completed.


In [43]:
#呃錢

In [44]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_呃錢.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711987964&srchtxt=%E5%91%83%E9%8C%A2&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E5%91%83%E9%8C%A2&&page=1"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
Finished scraping page 11.
No more topics found on page 12. Ending scrape.
Data scraping and CSV writing completed.


In [45]:
#欺詐

In [46]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_欺詐.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711989099&srchtxt=%E6%AC%BA%E8%A9%90&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E6%AC%BA%E8%A9%90&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
No more topics found on page 4. Ending scrape.
Data scraping and CSV writing completed.


In [47]:
#欺騙

In [48]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_欺騙.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711989441&srchtxt=%E6%AC%BA%E9%A8%99&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E6%AC%BA%E9%A8%99&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
No more topics found on page 11. Ending scrape.
Data scraping and CSV writing completed.


In [49]:
#詐騙

In [50]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_詐騙.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711990299&srchtxt=%E8%A9%90%E9%A8%99&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E8%A9%90%E9%A8%99&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
Finished scraping page 11.
Finished scraping page 12.
Finished scraping page 13.
No more topics found on page 14. Ending scrape.
Data scraping and CSV writing completed.


In [51]:
#電詐

In [52]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_電詐.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711991225&srchtxt=%E9%9B%BB%E8%A9%90&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%9B%BB%E8%A9%90&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [53]:
#電郵騙案

In [54]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_電郵騙案.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711991364&srchtxt=%E9%9B%BB%E9%83%B5%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%9B%BB%E9%83%B5%E9%A8%99%E6%A1%88&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [55]:
#電話騙案

In [56]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_電話騙案.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711991476&srchtxt=%E9%9B%BB%E8%A9%B1%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%9B%BB%E8%A9%B1%E9%A8%99%E6%A1%88&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
No more topics found on page 6. Ending scrape.
Data scraping and CSV writing completed.


In [57]:
#電騙

In [58]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_電騙.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711991900&srchtxt=%E9%9B%BB%E9%A8%99&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%9B%BB%E9%A8%99&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
No more topics found on page 3. Ending scrape.
Data scraping and CSV writing completed.


In [59]:
#網上情緣騙案

In [60]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_網上情緣騙案.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711992036&srchtxt=%E7%B6%B2%E4%B8%8A%E6%83%85%E7%B7%A3%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E7%B6%B2%E4%B8%8A%E6%83%85%E7%B7%A3%E9%A8%99%E6%A1%88&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [61]:
#網上詐騙

In [62]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_網上詐騙.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711992330&srchtxt=%E7%B6%B2%E4%B8%8A%E8%A9%90%E9%A8%99&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E7%B6%B2%E4%B8%8A%E8%A9%90%E9%A8%99&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [63]:
#網上騙案

In [64]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_網上騙案.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711992484&srchtxt=%E7%B6%B2%E4%B8%8A%E9%A8%99%E6%A1%88&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E7%B6%B2%E4%B8%8A%E9%A8%99%E6%A1%88&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [65]:
#騙徒

In [66]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_騙徒.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711992721&srchtxt=%E9%A8%99%E5%BE%92&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E5%BE%92&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
No more topics found on page 11. Ending scrape.
Data scraping and CSV writing completed.


In [67]:
#騙徒手法層出不窮

In [68]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_騙徒手法層出不窮.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711993452&srchtxt=%E9%A8%99%E5%BE%92%E6%89%8B%E6%B3%95%E5%B1%A4%E5%87%BA%E4%B8%8D%E7%AA%AE&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E5%BE%92%E6%89%8B%E6%B3%95%E5%B1%A4%E5%87%BA%E4%B8%8D%E7%AA%AE&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
No more topics found on page 2. Ending scrape.
Data scraping and CSV writing completed.


In [69]:
#騙錢

In [70]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_騙錢.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711993609&srchtxt=%E9%A8%99%E9%8C%A2&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E9%8C%A2&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
No more topics found on page 3. Ending scrape.
Data scraping and CSV writing completed.


In [71]:
#騙子

In [72]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_騙子.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711993715&srchtxt=%E9%A8%99%E5%AD%90&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E5%AD%90&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
Finished scraping page 11.
Finished scraping page 12.
Finished scraping page 13.
Finished scraping page 14.
Finished scraping page 15.
No more topics found on page 16. Ending scrape.
Data scraping and CSV writing completed.


In [ ]:
#騙局

In [73]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to make requests with retries
def make_request(url, headers, retries=5, delay=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed with error {e}. Retrying...")
            time.sleep(delay)
    return None  # If all retries fail, return None

# Function to scrape individual post details
def scrape_post_details(post_url, headers):
    response = make_request(post_url, headers)
    if response:
        post_soup = BeautifulSoup(response.content, 'html.parser')

        # Check if 'authi' class div is present before finding the 'a' tag
        authi_div = post_soup.find('div', class_='authi')
        if authi_div and authi_div.find('a'):
            username_tag = authi_div.find('a')
            username = username_tag.get_text(strip=True)
        else:
            username = "N/A"

        # Same check for the 'em' tag with 'authorposton' id before finding the 'span' tag
        authorposton_em = post_soup.find('em', id=lambda x: x and x.startswith('authorposton'))
        if authorposton_em and authorposton_em.find('span'):
            time_tag = authorposton_em.find('span')
            time = time_tag['title'] if 'title' in time_tag.attrs else "N/A"
        else:
            time = "N/A"

        # Extracting the main content
        pcb_div = post_soup.find('div', class_='pcb')
        if pcb_div and pcb_div.find('div', class_='t_fsz'):
            main_content_tag = pcb_div.find('div', class_='t_fsz')
            main_content = main_content_tag.get_text(strip=True)
        else:
            main_content = "N/A"

        # Extracting replies from all pages
        replies = []
        page_number = 1
        while True:
            # Find all reply divs
            reply_divs = post_soup.find_all('div', id=lambda x: x and x.startswith('postmessage_'))

            # Extract text from reply divs
            for reply_div in reply_divs:
                reply_text = reply_div.get_text(strip=True)
                replies.append(reply_text)

            # Find the next page button
            next_page_button = post_soup.find('i', class_='ico_on icon-pagin-generic_nextPageOn')

            # Break the loop if there's no next page button
            if not next_page_button:
                break

            # Increment page number and get the next page URL
            page_number += 1
            next_page_url = f"{post_url}&page={page_number}"
            
            # Make request for the next page
            response = make_request(next_page_url, headers)
            if response:
                post_soup = BeautifulSoup(response.content, 'html.parser')
            else:
                break  # Break loop if request fails

        return username, time, main_content, replies
    else:
        return "N/A", "N/A", "N/A", []

# CSV file setup
csv_file = "BK_騙局.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Replies/Views', 'Time', 'URL', 'Username', 'MainContent', 'Replies'])  # Writing the headers

    # Base URL pattern
    base_search_url = "https://www.baby-kingdom.com/search.php?mod=forum&searchid=1711993965&srchtxt=%E9%A8%99%E5%B1%80&orderby=lastpost&ascdesc=desc&searchsubmit=yes&keyword=%E9%A8%99%E5%B1%80&"

    # Scrape process
    page_number = 1
    while True:
        url = f"{base_search_url}&page={page_number}"
        page_response = make_request(url, headers)
        
        if page_response:
            soup = BeautifulSoup(page_response.content, 'html.parser')
            posts = soup.find_all('li', class_='pbw')
            
            if not posts:
                print(f"No more topics found on page {page_number}. Ending scrape.")
                break
            
            for post in posts:
                title_link = post.find('a')
                title = title_link.get_text(strip=True) if title_link else "N/A"
                post_url = title_link['href'] if title_link else "N/A"
                reply_view_element = post.find('p', class_='xg1')
                replies_views = reply_view_element.get_text(strip=True) if reply_view_element else "N/A"
                
                # Scrape details from the post's individual page
                if post_url.startswith('/'):
                    post_url = f"https://www.baby-kingdom.com{post_url}"
                username, post_time, main_content, all_replies = scrape_post_details(post_url, headers)
                
                # Flatten replies into a single string
                all_replies_str = ' | '.join(all_replies)
                
                writer.writerow([title, replies_views, post_time, post_url, username, main_content, all_replies_str])

            print(f"Finished scraping page {page_number}.")
            page_number += 1
        else:
            print(f"Failed to retrieve data for page {page_number}. Skipping...")
            page_number += 1
            continue

print("Data scraping and CSV writing completed.")


Finished scraping page 1.
Finished scraping page 2.
Finished scraping page 3.
Finished scraping page 4.
Finished scraping page 5.
Finished scraping page 6.
Finished scraping page 7.
Finished scraping page 8.
Finished scraping page 9.
Finished scraping page 10.
No more topics found on page 11. Ending scrape.
Data scraping and CSV writing completed.
